<a href="https://colab.research.google.com/github/sudheervobili/Pytorch/blob/main/question_answer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import pandas as pd

In [4]:
df = pd.read_csv('qa_dataset_500.csv')

In [5]:
df.head()

,question,answer
0,How many wheels does a bicycle have?,2
1,Who painted the Mona Lisa?,Leonardo da Vinci
2,Who wrote Romeo and Juliet?,William Shakespeare
3,How many letters are there in the English alph...,26
4,How many legs does a spider have?,8


In [6]:
df.shape

(500, 2)

In [7]:
def tokenize(text):
  text = text.lower()
  text = text.replace("?"," ")
  text = text.replace("."," ")
  text = text.replace(","," ")
  text = text.replace("!"," ")
  return text.split()

In [8]:
tokenize(df['question'][0])

['how', 'many', 'wheels', 'does', 'a', 'bicycle', 'have']

In [9]:
# vocab
vocab = {'<UNK>':0}

In [10]:
def build_vocab(row):
  tokenized_question = tokenize(row['question'])
  tokenized_answer = tokenize(row['answer'])

  merged_tokens = tokenized_question + tokenized_answer

  for token in merged_tokens:

    if token not in vocab:
      vocab[token] = len(vocab)


In [11]:
df.apply(build_vocab, axis=1)

,0
0,None
1,None
2,None
3,None
4,None
...,...
495,None
496,None
497,None
498,None


In [12]:
# convert words to numerical indices
def text_to_indices(text, vocab):

  indexed_text = []

  for token in tokenize(text):

    if token in vocab:
      indexed_text.append(vocab[token])
    else:
      indexed_text.append(vocab['<UNK>'])

  return indexed_text

In [13]:
text_to_indices("What is bicycle", vocab)

[33, 34, 6]

In [14]:
import torch
from torch.utils.data import Dataset, DataLoader

In [15]:
class QADataset(Dataset):

  def __init__(self, df, vocab):
    self.df = df
    self.vocab = vocab

  def __len__(self):
    return self.df.shape[0]

  def __getitem__(self, index):

    numerical_question = text_to_indices(self.df.iloc[index]['question'], self.vocab)
    numerical_answer = text_to_indices(self.df.iloc[index]['answer'], self.vocab)

    return torch.tensor(numerical_question), torch.tensor(numerical_answer)

In [16]:
dataset = QADataset(df, vocab)

In [17]:
dataloader = DataLoader(dataset, batch_size=1, shuffle=True)

In [18]:
for question, answer in dataloader:
  print(question, answer[0])

tensor([[ 33,  34,  11, 166, 171]]) tensor([172, 173])
tensor([[ 33, 153, 154, 155]]) tensor([156])
tensor([[ 33,  34,  11, 182, 183,  26,  11, 184, 185]]) tensor([186])
tensor([[1, 2, 3, 4, 5, 6, 7]]) tensor([8])
tensor([[ 9, 17, 18, 19, 20]]) tensor([21, 22])
tensor([[ 33, 143, 144, 145, 146, 147, 148]]) tensor([149])
tensor([[33, 41, 34, 42, 26, 43]]) tensor([44])
tensor([[ 45, 139,  34, 150, 151,  11,  90]]) tensor([152])
tensor([[ 33,  34,  11, 126, 174,  52, 175]]) tensor([176])
tensor([[  1,   2, 177,  24,  25,  26,   5, 178, 143]]) tensor([179])
tensor([[ 33, 153, 203, 204,  26]]) tensor([205])
tensor([[33, 34, 11, 50, 51, 52, 53]]) tensor([32])
tensor([[45, 54, 34, 55, 56, 11, 54, 52, 57]]) tensor([58])
tensor([[ 33, 153, 154, 155]]) tensor([156])
tensor([[ 33,  34,  11,  41,  52,  11, 169]]) tensor([170])
tensor([[ 33, 153, 154, 155]]) tensor([156])
tensor([[ 45, 105,  34,  55,  56,  11, 106,  52,  11, 107]]) tensor([108])
tensor([[ 9, 34, 11, 59, 52, 60, 61]]) tensor([62, 63

In [19]:
import torch.nn as nn

In [20]:
class SimpleRNN(nn.Module):

  def __init__(self, vocab_size):
    super().__init__()
    self.embedding = nn.Embedding(vocab_size, embedding_dim=50)
    self.rnn = nn.RNN(50, 64, batch_first=True)
    self.fc = nn.Linear(64, vocab_size)

  def forward(self, question):
    embedded_question = self.embedding(question)
    hidden, final = self.rnn(embedded_question)
    output = self.fc(final.squeeze(0))

    return output

In [21]:
x = nn.Embedding(324, embedding_dim=50)
y = nn.RNN(50, 64, batch_first=True)
z = nn.Linear(64, 324)

a = dataset[0][0].reshape(1,7)
print("shape of a:", a.shape)
b = x(a)
print("shape of b:", b.shape)
c, d = y(b)
print("shape of c:", c.shape)
print("shape of d:", d.shape)

e = z(d.squeeze(0))

print("shape of e:", e.shape)

shape of a: torch.Size([1, 7])
shape of b: torch.Size([1, 7, 50])
shape of c: torch.Size([1, 7, 64])
shape of d: torch.Size([1, 1, 64])
shape of e: torch.Size([1, 324])


In [22]:
learning_rate = 0.001
epochs = 30

In [23]:
model = SimpleRNN(len(vocab))

In [25]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [28]:
# training loop

for epoch in range(epochs):

  total_loss = 0

  for question, answer in dataloader:

    optimizer.zero_grad()

    # forward pass
    output = model(question)

    # Use only the first token of the answer as the target for loss calculation
    loss = criterion(output.squeeze(0), answer.squeeze(0)[0])


    # gradients
    loss.backward()

    # update
    optimizer.step()

    total_loss = total_loss + loss.item()

  print(f"Epoch: {epoch+1}, Loss: {total_loss:4f}")

Epoch: 1, Loss: 1380.460783
Epoch: 2, Loss: 200.924250
Epoch: 3, Loss: 48.378165
Epoch: 4, Loss: 20.891111
Epoch: 5, Loss: 11.606535
Epoch: 6, Loss: 7.294984
Epoch: 7, Loss: 4.901387
Epoch: 8, Loss: 3.432017
Epoch: 9, Loss: 2.472942
Epoch: 10, Loss: 1.818591
Epoch: 11, Loss: 1.354225
Epoch: 12, Loss: 1.020124
Epoch: 13, Loss: 0.774125
Epoch: 14, Loss: 0.590703
Epoch: 15, Loss: 0.453092
Epoch: 16, Loss: 0.348455
Epoch: 17, Loss: 0.268911
Epoch: 18, Loss: 0.208058
Epoch: 19, Loss: 0.161161
Epoch: 20, Loss: 0.125156
Epoch: 21, Loss: 0.097098
Epoch: 22, Loss: 0.075447
Epoch: 23, Loss: 0.058723
Epoch: 24, Loss: 0.045667
Epoch: 25, Loss: 0.035590
Epoch: 26, Loss: 0.027708
Epoch: 27, Loss: 0.021622
Epoch: 28, Loss: 0.016848
Epoch: 29, Loss: 0.013140
Epoch: 30, Loss: 0.010239


In [29]:
def predict(model, question, threshold=0.5):

  # convert question to numbers
  numerical_question = text_to_indices(question, vocab)

  # tensor
  question_tensor = torch.tensor(numerical_question).unsqueeze(0)

  # send to model
  output = model(question_tensor)

  # convert logits to probs
  probs = torch.nn.functional.softmax(output, dim=1)

  # find index of max prob
  value, index = torch.max(probs, dim=1)

  if value < threshold:
    print("I don't know")

  print(list(vocab.keys())[index])

In [32]:
predict(model, "who is sudheer")

yen


In [30]:
predict(model, "How many legs does a spider have?	")

8
